# Model validation altimetry

Note: this notebook requires the model output data

In [1]:
import matplotlib.pyplot as plt
import modelskill as ms
import helper as h

In [2]:
obs_fldr = "../observations/" 
# mr_fldr = "../output/"    
mr_fldr = "../output_sample/"     # use this if you have not downloaded the full data from Zenodo yet
mr_fn = [f"{mr_fldr}Area_2014_2018.dfsu",f"{mr_fldr}Area_2019_2022.dfsu"]
if "sample" in mr_fldr:
    mr_fn = [mr_fn[0]]
save_png = "sample" not in mr_fldr

In [3]:
altlist = h.get_altimetry_obs(quality=None)

In [4]:
ds = h.read_and_concatenate_ds(mr_fn,out_fname=f"{mr_fldr}Area_all.dfsu")

In [5]:
mr = ms.DfsuModelResult(ds[0], name="MIKE21")
mr

<DfsuModelResult>: MIKE21
Time: 2017-02-01 00:00:00 - 2017-02-08 00:00:00
Quantity: Surface Elevation [m]

In [6]:
cmps = []
match_t = "2022" # You can choose a specific time period here
for o in altlist:
    print(f"Processing {o.name}...")
    try:
        cmp = ms.match(obs=o.sel(time=match_t), mod=mr) 
        cmps.append(cmp.remove_bias())
    except Exception as e:
        print(e)    
cc = ms.ComparerCollection(cmps)

Processing 3b...
No time overlap!
Processing c2...
No time overlap!
Processing j2...
"not all values found in index 'time'. Try setting the `method` keyword argument (example: method='nearest')."
Processing sa...
No time overlap!


c:\Users\frtp\AppData\Local\miniforge3\envs\WBOresund\Lib\site-packages\modelskill\model\_base.py:70: UserWarning: No time overlap. Observation '3b' outside model time range! 
  warnings.warn(
c:\Users\frtp\AppData\Local\miniforge3\envs\WBOresund\Lib\site-packages\modelskill\model\_base.py:70: UserWarning: No time overlap. Observation 'c2' outside model time range! 
  warnings.warn(
c:\Users\frtp\AppData\Local\miniforge3\envs\WBOresund\Lib\site-packages\modelskill\model\_base.py:70: UserWarning: No time overlap. Observation 'sa' outside model time range! 
  warnings.warn(


<ComparerCollection>
Comparers:

In [9]:
if len(cc)>0:
    cc.skill().round(4)
else:
    print("No data to show")

No data to show


In [10]:
if len(cc)>0:
    cc.plot.scatter(skill_table=True, figsize=(5,4));
    plt.title("MIKE21 vs Altimetry, "+match_t)
    if save_png: plt.savefig("../figures/altimetry_comparison.png", bbox_inches="tight")
else:
    print("No data to show")


No data to show


In [11]:
if len(cc)>0:
    fig, axs = plt.subplots(-(-len(cc)//2), 2, figsize=(8,len(cc)*2))
    axs = axs.flatten()
    for i in range(len(cc)):
        cc[i].sel().plot.scatter(ax=axs[i])

    if len(cc) % 2 != 0:
        axs[-1].axis("off")
    plt.tight_layout()
    plt.show()
else:
    print("No data to show")


No data to show
